In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current sessionOUR_HF_API_KEY

/kaggle/input/nlp-collected-data/code_civil_chunks_complete-unified.json
/kaggle/input/nlp-collected-data/jo_2021_chunks.json
/kaggle/input/nlp-collected-data/jo_2023_chunks.json
/kaggle/input/nlp-collected-data/jo_2020_chunks.json
/kaggle/input/nlp-collected-data/jo_2025_chunks.json
/kaggle/input/nlp-collected-data/jo_2024_chunks.json
/kaggle/input/nlp-collected-data/code_penale-unified.json
/kaggle/input/nlp-collected-data/jo_2022_chunks.json
/kaggle/input/nlp-collected-data/french_code_famille-unified.json
/kaggle/input/nlp-collected-data/french_commerce_chunks-unified.json
/kaggle/input/nlp-collected-data/constitution-unified.json


In [2]:
!pip install -q transformers torch sentence-transformers faiss-cpu accelerate bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 74.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 31.9 MB/s eta 0:00:00:00:0100:01


In [3]:
"""
Import all necessary libraries
"""
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from typing import List, Dict, Tuple
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")
print(f"✓ PyTorch version: {torch.__version__}")
print(f"✓ CUDA available: {torch.cuda.is_available()}")

2025-12-22 16:24:02.429942: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766420642.622187      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766420642.677467      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766420643.135584      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766420643.135625      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766420643.135628      55 computation_placer.cc:177] computation placer alr

✓ All libraries imported successfully!
✓ PyTorch version: 2.8.0+cu126
✓ CUDA available: True


In [4]:
# ============================================================================
"""
Set your configuration parameters here
"""
# Paths
JSON_FOLDER = "/kaggle/input/nlp-collected-data"  # Change this to your dataset path
INDEX_PATH = "algerian_legal(jo+constitution+penale+civil+commerce+famille) embedder: dangvantuan/sentence-camembert-large.faiss"

# Models
EMBEDDING_MODEL = "dangvantuan/sentence-camembert-large"  # Best for French
LLM_MODEL = "bofenghuang/vigogne-2-7b-chat"  # 7B French model

# Settings
USE_4BIT_QUANTIZATION = True  # Set False if you have lots of RAM
TOP_K_RETRIEVAL = 3  # Number of documents to retrieve

print("✓ Configuration set!")
print(f"  JSON Folder: {JSON_FOLDER}")
print(f"  Embedding Model: {EMBEDDING_MODEL}")
print(f"  LLM Model: {LLM_MODEL}")

✓ Configuration set!
  JSON Folder: /kaggle/input/nlp-collected-data
  Embedding Model: dangvantuan/sentence-camembert-large
  LLM Model: bofenghuang/vigogne-2-7b-chat


In [5]:
"""
Load the embedding model for document vectorization
"""
print("Loading embedding model...")
embedding_model = SentenceTransformer(EMBEDDING_MODEL)
embedding_dimension = embedding_model.get_sentence_embedding_dimension()

print(f"✓ Embedding model loaded!")
print(f"  Model: {EMBEDDING_MODEL}")
print(f"  Dimension: {embedding_dimension}")

Loading embedding model...


config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/809k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/298 [00:00<?, ?B/s]

✓ Embedding model loaded!
  Model: dangvantuan/sentence-camembert-large
  Dimension: 1024


In [6]:
"""
Load the language model for answer generation
"""
print("Loading LLM for generation...")

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)

if USE_4BIT_QUANTIZATION and torch.cuda.is_available():
    # 4-bit quantization for memory efficiency
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4"
    )
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True
    )
    print(f"✓ LLM loaded with 4-bit quantization!")
else:
    llm_model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto" if torch.cuda.is_available() else None,
        trust_remote_code=True
    )
    print(f"✓ LLM loaded!")

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"  Device: {device}")
print(f"  Model: {LLM_MODEL}")


Loading LLM for generation...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

...

pytorch_model-00001-of-00007.bin:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

✓ LLM loaded with 4-bit quantization!
  Device: cuda
  Model: bofenghuang/vigogne-2-7b-chat


In [7]:
"""
Load all JSON files from the folder
Expected format:
{
    "source_document_type": "constitution",
    "header": "Title I - Chapter PREMIER - Article 1er",
    "content": "L'Algérie est une République..."
}
"""
print("Loading documents from JSON files...")

documents = []
path = Path(JSON_FOLDER)

if path.is_dir():
    json_files = sorted(list(path.glob("*.json")))
    print(f"Found {len(json_files)} JSON files")
    
    for i, json_file in enumerate(json_files, 1):
        print(f"  [{i}/{len(json_files)}] Loading {json_file.name}...", end=" ")
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if isinstance(data, dict):
            documents.append(data)
            print("✓ (1 document)")
        elif isinstance(data, list):
            documents.extend(data)
            print(f"✓ ({len(data)} documents)")

elif path.is_file():
    print(f"Loading single file: {JSON_FOLDER}")
    with open(JSON_FOLDER, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    if isinstance(data, dict):
        documents = [data]
    elif isinstance(data, list):
        documents = data

print(f"\n✓ Total documents loaded: {len(documents)}")

# Show sample
if documents:
    sample = documents[0]
    print("\nSample document:")
    print(f"  Type: {sample.get('source_document_type', 'N/A')}")
    print(f"  Header: {sample.get('header', 'N/A')[:60]}...")
    print(f"  Content: {sample.get('content', 'N/A')[:100]}...")


Loading documents from JSON files...
Found 11 JSON files
  [1/11] Loading code_civil_chunks_complete-unified.json... ✓ (1466 documents)
  [2/11] Loading code_penale-unified.json... ✓ (335 documents)
  [3/11] Loading constitution-unified.json... ✓ (219 documents)
  [4/11] Loading french_code_famille-unified.json... ✓ (34 documents)
  [5/11] Loading french_commerce_chunks-unified.json... ✓ (915 documents)
  [6/11] Loading jo_2020_chunks.json... ✓ (2373 documents)
  [7/11] Loading jo_2021_chunks.json... ✓ (3190 documents)
  [8/11] Loading jo_2022_chunks.json... ✓ (3246 documents)
  [9/11] Loading jo_2023_chunks.json... ✓ (2993 documents)
  [10/11] Loading jo_2024_chunks.json... ✓ (2045 documents)
  [11/11] Loading jo_2025_chunks.json... ✓ (1906 documents)

✓ Total documents loaded: 18722

Sample document:
  Type: code_civil
  Header: DISPOSITIONS GENERALES - DES EFFETS ET DE L'APPLICATION DES ...
  Content: Article 1 . - La loi régit toutes les matières auxquelles se rapporte la lettre ou

In [8]:
# CELL 6.5: Additional Cleaning (if needed)
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'  # For better error messages

# Filter out very short or problematic documents
documents = [
    doc for doc in documents 
    if doc.get('content') and len(doc.get('content', '').strip()) > 10
]
print(f"Filtered to {len(documents)} valid documents")

Filtered to 18722 valid documents


In [9]:
"""
Convert all documents to vector embeddings
"""
import re

print("Creating embeddings for all documents...")

def clean_text(text):
    """Clean text to avoid CUDA errors"""
    if not text:
        return ""
    # Remove null bytes and other problematic characters
    text = text.replace('\x00', '').replace('\ufffd', '')
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    # Limit length to avoid memory issues
    max_length = 512  # Characters, not tokens
    if len(text) > max_length:
        text = text[:max_length]
    return text.strip()

texts = []
for i, doc in enumerate(documents):
    try:
        # Combine metadata and content
        doc_type = doc.get('source_document_type', '').upper()
        header = doc.get('header', '')
        content = doc.get('content', '')
        
        # Clean all text components
        doc_type = clean_text(doc_type)
        header = clean_text(header)
        content = clean_text(content)
        
        # Format: [TYPE] Header: Content
        full_text = f"[{doc_type}] {header}\n{content}"
        
        # Final validation
        if full_text and len(full_text.strip()) > 0:
            texts.append(full_text)
        else:
            print(f"  Warning: Skipping empty document at index {i}")
            
    except Exception as e:
        print(f"  Error processing document {i}: {e}")
        continue

print(f"✓ Prepared {len(texts)} documents for encoding")

# Encode in smaller batches to avoid CUDA errors
print(f"Encoding documents...")
batch_size = 16  # Reduced batch size for stability
all_embeddings = []

for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i+batch_size]
    print(f"  Processing batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}...", end=" ")
    
    try:
        batch_embeddings = embedding_model.encode(
            batch_texts,
            batch_size=batch_size,
            show_progress_bar=False,
            convert_to_numpy=True,
            normalize_embeddings=True
        )
        all_embeddings.append(batch_embeddings)
        print("✓")
        
    except Exception as e:
        print(f"✗ Error: {e}")
        # Skip problematic batch or try with smaller size
        print(f"  Retrying with batch size 1...")
        for text in batch_texts:
            try:
                single_embedding = embedding_model.encode(
                    [text],
                    batch_size=1,
                    show_progress_bar=False,
                    convert_to_numpy=True,
                    normalize_embeddings=True
                )
                all_embeddings.append(single_embedding)
            except:
                print(f"  Failed to encode text: {text[:50]}...")
                continue

# Combine all embeddings
embeddings = np.vstack(all_embeddings)

print(f"\n✓ Embeddings created!")
print(f"  Shape: {embeddings.shape}")
print(f"  Data type: {embeddings.dtype}")

# Update documents list to match embeddings (in case some were skipped)
if len(embeddings) != len(documents):
    print(f"  Note: {len(documents) - len(embeddings)} documents were skipped")
    # Keep only successfully embedded documents
    documents = [documents[i] for i in range(min(len(documents), len(embeddings)))]



Creating embeddings for all documents...
✓ Prepared 18722 documents for encoding
Encoding documents...
  Processing batch 1/1171... ✓
  Processing batch 2/1171... ✓
  Processing batch 3/1171... ✓
  Processing batch 4/1171... ✓
  Processing batch 5/1171... ✓
  Processing batch 6/1171... ✓
  Processing batch 7/1171... ✓
  Processing batch 8/1171... ✓
  Processing batch 9/1171... ✓
  Processing batch 10/1171... ✓
  Processing batch 11/1171... ✓
  Processing batch 12/1171... ✓
  Processing batch 13/1171... ✓
  Processing batch 14/1171... ✓
  Processing batch 15/1171... ✓
  Processing batch 16/1171... ✓
  Processing batch 17/1171... ✓
  Processing batch 18/1171... ✓
  Processing batch 19/1171... ✓
  Processing batch 20/1171... ✓
  Processing batch 21/1171... ✓
  Processing batch 22/1171... ✓
  Processing batch 23/1171... ✓
  Processing batch 24/1171... ✓
  Processing batch 25/1171... ✓
  Processing batch 26/1171... ✓
  Processing batch 27/1171... ✓
  Processing batch 28/1171... ✓
  Processi

In [10]:
"""
Build FAISS index for fast similarity search
"""
print("Building FAISS index...")

# Create index (using cosine similarity with normalized embeddings)
index = faiss.IndexFlatIP(embedding_dimension)
index.add(embeddings.astype('float32'))

print(f"✓ FAISS index built!")
print(f"  Total vectors: {index.ntotal}")
print(f"  Dimension: {embedding_dimension}")


Building FAISS index...
✓ FAISS index built!
  Total vectors: 18722
  Dimension: 1024


In [12]:
INDEX_PATH = "algerian_legal(jo+constitution+penale+civil+commerce+famille) embedder: dangvantuan-sentence-camembert-large.faiss"


In [13]:
"""
Save FAISS index, documents, and metadata to disk
"""
print("Saving index and documents...")

# Save FAISS index
faiss.write_index(index, INDEX_PATH)
print(f"✓ FAISS index saved: {INDEX_PATH}")

# Save documents
docs_path = INDEX_PATH.replace('.faiss', '_docs.json')
with open(docs_path, 'w', encoding='utf-8') as f:
    json.dump(documents, f, ensure_ascii=False, indent=2)
print(f"✓ Documents saved: {docs_path}")

# Save metadata
meta_path = INDEX_PATH.replace('.faiss', '_meta.json')
metadata = {
    "num_documents": len(documents),
    "embedding_model": EMBEDDING_MODEL,
    "llm_model": LLM_MODEL,
    "dimension": embedding_dimension,
    "document_types": list(set(d.get('source_document_type', '') for d in documents))
}
with open(meta_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, ensure_ascii=False, indent=2)
print(f"✓ Metadata saved: {meta_path}")

print("\n✓ All files saved successfully!")


Saving index and documents...
✓ FAISS index saved: algerian_legal(jo+constitution+penale+civil+commerce+famille) embedder: dangvantuan-sentence-camembert-large.faiss
✓ Documents saved: algerian_legal(jo+constitution+penale+civil+commerce+famille) embedder: dangvantuan-sentence-camembert-large_docs.json
✓ Metadata saved: algerian_legal(jo+constitution+penale+civil+commerce+famille) embedder: dangvantuan-sentence-camembert-large_meta.json

✓ All files saved successfully!


In [14]:
"""
Function to retrieve relevant documents
"""
def retrieve_documents(query: str, top_k: int = 3):
    """
    Retrieve most relevant documents for a query
    
    Args:
        query: User's question
        top_k: Number of documents to retrieve
        
    Returns:
        List of (document, score) tuples
    """
    # Encode query
    query_embedding = embedding_model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    
    # Search index
    scores, indices = index.search(query_embedding.astype('float32'), top_k)
    
    # Return documents with scores
    results = []
    for idx, score in zip(indices[0], scores[0]):
        if idx < len(documents):
            results.append((documents[idx], float(score)))
    
    return results

print("✓ Retrieval function defined!")

✓ Retrieval function defined!


In [15]:
"""
Function to generate answers using LLM
"""
def generate_answer(query: str, context_docs: List[Tuple[Dict, float]]) -> str:
    """
    Generate answer using retrieved context
    
    Args:
        query: User's question
        context_docs: Retrieved documents with scores
        
    Returns:
        Generated answer
    """
    # Build context
    context_parts = []
    for doc, score in context_docs:
        doc_type = doc.get('source_document_type', '').upper()
        header = doc.get('header', '')
        content = doc.get('content', '')
        
        ctx = f"[{doc_type}] {header}\n{content}"
        context_parts.append(ctx)
    
    context = "\n\n".join(context_parts)
    
    # Create prompt
    prompt = f"""<|system|>: Tu es un assistant juridique expert en droit algérien. Réponds de manière précise, professionnelle et factuelle en te basant strictement sur le contexte fourni. Si l'information n'est pas dans le contexte, indique-le clairement.
<|user|>: Contexte juridique:
{context}

Question: {query}
<|assistant|>:"""
    
    # Tokenize
    inputs = tokenizer(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}
    
    # Generate
    with torch.no_grad():
        outputs = llm_model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.7,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1
        )
    
    # Decode
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract answer
    if "<|assistant|>:" in response:
        answer = response.split("<|assistant|>:")[-1].strip()
    else:
        answer = response[len(prompt):].strip()
    
    return answer

print("✓ Generation function defined!")

✓ Generation function defined!


In [16]:
"""
Complete RAG query function
"""
def query_rag(question: str, top_k: int = 3, show_sources: bool = True):
    """
    Complete RAG query
    
    Args:
        question: User's question
        top_k: Number of documents to retrieve
        show_sources: Whether to display sources
        
    Returns:
        Dictionary with answer and sources
    """
    print(f"\n{'='*70}")
    print(f"Question: {question}")
    print('='*70)
    
    # Retrieve documents
    print("\n🔍 Retrieving relevant documents...")
    retrieved_docs = retrieve_documents(question, top_k)
    
    # Generate answer
    print("🤖 Generating answer...")
    answer = generate_answer(question, retrieved_docs)
    
    # Display results
    print(f"\n📝 Answer:\n{answer}\n")
    
    if show_sources:
        print("📚 Sources:")
        for i, (doc, score) in enumerate(retrieved_docs, 1):
            print(f"\n  [{i}] {doc.get('source_document_type', '').upper()}")
            print(f"      {doc.get('header', '')}")
            print(f"      Relevance: {score:.4f}")
            content = doc.get('content', '')
            preview = content[:200] + "..." if len(content) > 200 else content
            print(f"      {preview}")
    
    return {
        "answer": answer,
        "sources": [(doc, score) for doc, score in retrieved_docs]
    }

print("✓ Complete query function defined!")


✓ Complete query function defined!


In [17]:
"""
Test the RAG system with a query about the constitution
"""
result1 = query_rag(
    "Qu'est-ce que l'Algérie selon la constitution?",
    top_k=3
)



Question: Qu'est-ce que l'Algérie selon la constitution?

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
Selon la Constitution algérienne, l'Algérie est une république démocratique et populaire, soit une nation gouvernée par des citoyens ayant le pouvoir de voter et participer à la vie politique du pays. L'article 1er de la Constitution stipule également qu'elle est "une et indivisible", ce qui signifie que sa souveraineté ne peut être partagée ou divisée entre plusieurs entités. En outre, la capitale de la République est Alger, comme mentionné dans l'article 5 de la Constitution.

📚 Sources:

  [1] CONSTITUTION
      Title I - Chapter PREMIER - Article 1er
      Relevance: 0.4948
      L'Algérie est une République Démocratique et Populaire. Elle est une et indivisible.

  [2] CONSTITUTION
      Title I - Chapter PREMIER - Article 5
      Relevance: 0.4773
      La capitale de la République est Alger.

  [3] CODE_CIVIL
      EUXIEME DE LA REPRESENTATION LEGALE - 

In [18]:
"""
Test with another query
"""
result2 = query_rag(
    "Quels sont les droits et libertés fondamentaux?",
    top_k=3
)


Question: Quels sont les droits et libertés fondamentaux?

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
Les droits et libertés fondamentaux mentionnés dans le contexte fourni sont :

1. La liberté de pensée et de conscience (art. 34 de la Constitution algérienne)
2. La liberté de religion et de conviction (art. 34 de la Constitution algérienne)
3. Le droit à la vie privée (art. 34 de la Constitution algérienne)
4. L'inviolabilité de la maison et de l'intimité familiale (art. 34 de la Constitution algérienne)
5. Le droit à l'intégrité physique (art. 34 de la Constitution algérienne)
6. Le droit à la liberté personnelle et individuelle (art. 34 de la Constitution algérienne)
7. Le droit à la liberté de mouvement et de résidence (art. 34 de la Constitution algérienne)
8. Le droit à la liberté d'expression et de réunion pacifique (art. 34 de la Constitution algérienne)
9. Le droit à la liberté de presse (art. 34 de la Constitution algérienne)
10. Le droit de grève 

In [19]:
"""
Test with a third query
"""
result3 = query_rag(
    "Expliquez les mesures de sûreté prévues par le code pénal",
    top_k=3
)


Question: Expliquez les mesures de sûreté prévues par le code pénal

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
Les mesures de sûreté prévues par le Code pénal algérien sont des dispositions légales visant à assurer la protection du public contre les individus susceptibles de commettre des actes contraires aux lois. Voici une explication détaillée de ces mesures :

1. Mesures de sûreté personnelles : Ce sont des restrictions imposées à un individu pour éviter qu'il ne puisse commettre des crimes ou délits. Ces mesures comprennent des obligations telles que l'obligation de ne pas se trouver dans des endroits spécifiques, l'interdiction de posséder certaines objets ou substances, etc.

2. Mesures de sûreté réelles : Ce sont des restrictions qui concernent un lieu spécifique où il est interdit d'accéder à des personnes, comme une prison ou un centre de détention.

Les mesures de sûreté peuvent être appliquées par le juge, en fonction des circonstances de l'affai

In [20]:
"""
Run your own custom queries here
Change the question variable to test different queries
"""
# Your custom question
my_question = "c'est quoi la religion en algerie"

# Run query
result = query_rag(my_question, top_k=3)


Question: c'est quoi la religion en algerie

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
La religion officielle en Algérie est l'islam, comme mentionné dans la constitution (Article 2). Cette déclaration fait partie du titre I - Chapitre Premier intitulé "l'Islam est la religion de l'Etat" de la Constitution algérienne. De plus, il existe une disposition concernant la liberté d'expression et la liberté de culte (Article 51) qui protège les droits individuels et garantit que les citoyens ont la liberté d'expression et de pratiquer leur culte sans interférence de la part de l'État. Les lois prévoient également que les lieux de culte soient protégés contre toute influence politique ou idéologique (Article 20-251, Journal officiel n° 36 du 15 septembre 2020).

📚 Sources:

  [1] CONSTITUTION
      Title I - Chapter PREMIER - Article 2
      Relevance: 0.3827
      L'Islam est la religion de l'Etat.

  [2] CONSTITUTION
      Title II - Chapter PREMIER - Article 51
 

In [21]:
"""
Test multiple questions at once
"""
questions = [
    "Quelle est la forme de l'État algérien?",
    "Quels sont les symboles de la République?",
    "Comment est organisé le pouvoir législatif?"
]

results = []
for question in questions:
    result = query_rag(question, top_k=2, show_sources=False)
    results.append(result)
    print("\n" + "="*70 + "\n")


Question: Quelle est la forme de l'État algérien?

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
La forme de l'État algérien est une République démocratique et populaire, comme stipulé dans la Constitution du pays.




Question: Quels sont les symboles de la République?

🔍 Retrieving relevant documents...
🤖 Generating answer...

📝 Answer:
Selon la Constitution et la loi numéro [JO_2020], les symboles de la République algérienne sont :

1. L'emblème national : Il représente les couleurs du pays (vert et blanc) avec une étoile et un croissant rouges au centre. Cet emblème est associé à la Révolution du 1er novembre 1954 et est considéré comme immutable.

2. L'hymne national : Il s'agit de "Qassaman", qui est le refrain complet du hymne national sans modification. Cette composition musicale est également reconnue comme immobile et fait partie des symboles de la République depuis sa création lors de la Révolution du 1er novembre 1954.




Question: Comment est organ

In [ ]:
"""
Use this cell to load a previously saved index
Skip cells 6-9 if you're loading an existing index
"""
# Uncomment to use:
"""
print("Loading existing index...")

# Load FAISS index
index = faiss.read_index(INDEX_PATH)
print(f"✓ FAISS index loaded: {index.ntotal} vectors")

# Load documents
docs_path = INDEX_PATH.replace('.faiss', '_docs.json')
with open(docs_path, 'r', encoding='utf-8') as f:
    documents = json.load(f)
print(f"✓ Documents loaded: {len(documents)} documents")

# Load metadata
meta_path = INDEX_PATH.replace('.faiss', '_meta.json')
with open(meta_path, 'r', encoding='utf-8') as f:
    metadata = json.load(f)
print(f"✓ Metadata loaded")
print(f"  Document types: {', '.join(metadata['document_types'])}")
"""

In [ ]:
"""
Export query results to a file
"""
# Uncomment to use:
"""
import pandas as pd

# Create results dataframe
results_data = []
for question in questions:
    result = query_rag(question, top_k=3, show_sources=False)
    results_data.append({
        'question': question,
        'answer': result['answer'],
        'num_sources': len(result['sources'])
    })

df = pd.DataFrame(results_data)
df.to_csv('rag_results.csv', index=False, encoding='utf-8')
print("✓ Results exported to rag_results.csv")
"""

In [22]:
"""
Display statistics about your RAG system
"""
print("=" * 70)
print("RAG SYSTEM STATISTICS")
print("=" * 70)
print(f"Total Documents: {len(documents)}")
print(f"Embedding Dimension: {embedding_dimension}")
print(f"FAISS Index Size: {index.ntotal} vectors")
print(f"Document Types: {set(d.get('source_document_type', '') for d in documents)}")
print(f"Embedding Model: {EMBEDDING_MODEL}")
print(f"LLM Model: {LLM_MODEL}")
print(f"Device: {'GPU' if torch.cuda.is_available() else 'CPU'}")
print("=" * 70)

RAG SYSTEM STATISTICS
Total Documents: 18722
Embedding Dimension: 1024
FAISS Index Size: 18722 vectors
Document Types: {'jo_2020', 'code_civil', 'jo_2023', 'jo_2021', 'jo_2022', 'jo_2024', 'jo_2025', 'code_penale', 'constitution', 'code_commerce', 'code_famille'}
Embedding Model: dangvantuan/sentence-camembert-large
LLM Model: bofenghuang/vigogne-2-7b-chat
Device: GPU
